In [4]:
import sys
import os
import pandas as pd

from bokeh.palettes import Category10


from file_cache.utils.util_pandas import *
from file_cache.cache import file_cache

import matplotlib.pyplot as plt

#Adjust the working folder
file_folder = globals()['_dh'][0]
wk_dir = os.path.dirname(file_folder)
os.chdir(wk_dir)
from core.feature import *
from core.train import *
logging.getLogger().setLevel(logging.DEBUG)



In [28]:
train = get_train_ex(1)
train.iloc[:, 0].iat[0]

Timestamp('2018-07-01 00:00:00.449000')

In [17]:

train.loc[:, 'var016'].value_counts()#.iloc[:2].sum()/len(train)
 

0.0     341827
11.0      3664
21.0      2347
6.0       1201
22.0       642
9.0        547
16.0       455
20.0       131
10.0        90
19.0        18
2.0         18
Name: var016, dtype: int64

In [ ]:

for wtid in range(1,2):
    group_list = group_columns(wtid=1)
    for group in group_list:
        logger.debug(f'Current group:{group}')
        train = get_train_ex(wtid)
        train = train[group]   
        logger.debug(f'Old len = {len(train)}')
        train = train.dropna(how='all')
        logger.debug(f'New len = {len(train)}')
        print(len(train), train.where(pd.isna(train), 1).sum())

    

In [ ]:
logging.getLogger().setLevel(logging.WARNING)
train_list = []
from tqdm import tqdm
for wtid in tqdm(range(1, 3)):
    train_ex =  predict_wtid(wtid)
    train_ex = train_ex.set_index(['ts', 'wtid'])
    train_list.append(train_ex)
submit = pd.concat(train_list)

In [ ]:
get_closed_columns('var034')

In [ ]:
logging.getLogger().setLevel(logging.INFO)
sub = predict_all()
sub.head()

## Analysis the std of the column

In [ ]:
tmp = get_std_all()

tmp.groupby(['col', 'data_type']).agg({'mean':['mean','min', 'max', 'std']}).sort_values(('mean','mean'), ascending=False)




## Visual missing block

In [ ]:
import matplotlib.pyplot as plt
for wtid in range(1, 5):
    block_list = get_blocks()
    block_list = block_list.loc[(block_list.wtid==wtid) &(block_list.kind=='missing') ]
    block_list

    plt.figure(figsize=(20,10))


    for col_name in block_list.col.drop_duplicates().sort_values().values:
        #print(col_name)
        block_sub = block_list.loc[block_list['col'] == col_name]

        for index, block in block_sub.iterrows():
            sn = int(block['col'][-3:])
            plt.plot([block.begin, block.end], [sn, sn], color=Category10[10][sn%10] )


plt.show()

## For special col

In [ ]:
import matplotlib.pyplot as plt
block = adjust_block(ratio = 1)
col_name = 'var011'
for wtid in range(1, 5):
    block_list = adjust_block()
    block_list = block_list.loc[(block_list.wtid==wtid) &(block_list.kind=='missing') ]
    block_list

    plt.figure(figsize=(20,3))

    count = 0

    for col_name in get_closed_columns(col_name,threshold=0.8):
        #print(col_name)
        block_sub = block_list.loc[block_list['col'] == col_name].reset_index()
        
        count += 1
        for index, block in block_sub.iterrows():
            #sn = int(block['col'][-3:])
            plt.plot([block.begin_ex, block.end_ex], [count, count], color=Category10[10][index%10] )

plt.show()



In [ ]:

block2 = block[(block.wtid < 5) &(block.col=='var003') & (block.kind=='missing')]
block2.end_ex.max(), block2.end.max()
#block.loc[(block.wtid==1) &(block.col=='var001')&(block.kind=='missing')]




## Main

In [ ]:
block_list = get_blocks()
for wtid in range(1, 2):
    train_ex = get_train_ex(wtid)
    for blockid, missing_block in block_list.loc[(block_list.wtid == wtid) & (block_list.kind == 'missing')].iterrows():
        col_name = missing_block.col
        train, sub = get_submit_feature_by_block_id(blockid)
        logger.info(f'blockid:{blockid}, train:{train.shape}')
        predict_fn = get_predict_fun(blockid, train)
        predict_res = predict_fn(sub.time_sn)
        sub[col_name] = predict_res
        logger.debug(f'predict_res:{predict_res.shape}, {type(predict_res)}')
        begin, end = missing_block.begin, missing_block.end

        logger.debug(f'train.loc[begin:end,col_name] = {train_ex.loc[begin:end,col_name].shape}, {begin}, {end}, {wtid}, {col_name}')
        train_ex.loc[begin:end,col_name] = predict_res
        
train_ex

## Get corrcoef matrix

In [ ]:
sub = get_train_ex(1)

sub = sub.dropna(how='any')

cor = np.corrcoef(sub.drop(axis='column', columns=['ts','wtid']).T)

col_list = sub.columns[2:]

print(cor.shape, sub.shape)

cor1 = pd.DataFrame(index=col_list, columns=col_list, data=np.where(abs(cor)>0, cor, 0) )

cor1 

In [ ]:
sub = get_train_ex(3)

sub = sub.dropna(how='any')

cor = np.corrcoef(sub.drop(axis='column', columns=['ts','wtid']).T)

col_list = sub.columns[2:]

print(cor.shape, sub.shape)

cor2 = pd.DataFrame(index=col_list, columns=col_list, data=np.where(abs(cor)>0, cor, 0) )
cor2
abs(cor2 - cor1)
cor_all = pd.DataFrame(index=col_list, columns=col_list, data = np.where(abs(abs(cor2) - abs(cor1)) > 0.2 , 1, 0) )
cor_all

## VIsual predict result

In [ ]:

logging.getLogger().setLevel(logging.INFO)
import matplotlib.pyplot as plt
col = 'var034'
train_list = get_train_feature(3, col)

train_list = sorted(train_list, key=lambda val: len(val[1]), reverse=True)

count, loss = 0, 0
for train, val, blockid in train_list[:10]:
    plt.figure(figsize=(20,5))
    is_enum = True if 'int' in date_type[col].__name__ else False
    
#     if is_enum:
#         fn = lambda val:  np.full_like(val,train[col].value_counts().index[0])
#     else:    
#         w = np.polyfit (train.time_sn, train[col], 1)
#         fn = np.poly1d(w)
    fn = get_predict_fun(blockid,train)
    
    for color, data in zip(Category10[5][:2], [train, val]):
        plt.scatter(data.time_sn, data[col], c= color)
    
    x = np.linspace(train.time_sn.min(), train.time_sn.max(), 10000)
    
    
    cur_count, cur_loss = score(val[col], fn(val.time_sn), is_enum)
    
    loss  += cur_loss
    count += cur_count
    
    plt.plot(x, fn(x))    
    #plt.plot(x, np.full_like(x, train[col].mean()) , c='r')  
    plt.show()
    logger.info(f'blockid:{blockid}, {train.shape}, {val.shape}, score={round(cur_loss/cur_count,3)}')
print(f'Total loss:{round(loss/count, 4)}, is_enum:{is_enum}')

## Predict submit data

In [ ]:


block = get_blocks() 
wtid = 1
col = 'var001'
missing_block = block.loc[(block.wtid == wtid) & (block.col == col) & (block.kind == 'missing')]
missing_block
for blockid, row in missing_block.iterrows():
    train, sub = get_submit_feature_by_block_id(blockid)
    logger.info(f'blockid:{blockid}, train:{train.shape}')
    predict_fn = get_predict_fun(blockid, train)
    predict_res = predict_fn(sub.time_sn)
    sub[col] = predict_res
    
    for color, data in zip(Category10[5][:2], [train, sub]):
        plt.scatter(data.time_sn, data[col], c= color)
    
    x = np.linspace(train.time_sn.min(), train.time_sn.max(), 10000)
    
    
    cur_count, cur_loss = score(sub[col], fn(sub.time_sn), is_enum)
    
    loss  += cur_loss
    count += cur_count
    
    plt.plot(x, predict_fn(x))    
#     plt.plot(x, np.full_like(x, train[col].mean()) , c='r')  
    plt.show()

## Compare diff wtid


In [2]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,5))


get_corr_wtid('var043')


2019-01-26 18:25:42,799 util_log.py[41] INFO get_corr_wtid begin with(1 paras) :['var043'], []
2019-01-26 18:25:42,804 cache.py[38] DEBUG Can not find cache from file:./cache/get_corr_wtid=var043=.h5
2019-01-26 18:25:47,381 util_log.py[41] INFO get_sub_template begin with(0 paras) :[], []
2019-01-26 18:25:47,561 cache.py[29] DEBUG Read cache from file:./cache/get_sub_template==.h5,key:['/df_0']
2019-01-26 18:25:47,987 util_log.py[49] INFO get_sub_template cost    0.61 sec:(0 paras)([], []), return:DataFrame, end 
/Users/lali2/Documents/workspace_py/df_jf/core/feature.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  template.drop(columns='sn', errors='ignore', inplace=True)
2019-01-26 18:25:49,272 feature.py[96] DEBUG Convert train#1 from (352124, 70) to (359846, 69)
/Users/lali2/Documents/workspace_py/df_jf/core/fe

2019-01-26 18:29:59,268 feature.py[494] DEBUG    var043_1  var043_2  var043_3  var043_4  var043_5  var043_6  var043_7  var043_8  var043_9  var043_10  var043_11  var043_12  var043_13  var043_14  var043_15  var043_16  var043_17  var043_18  var043_19  var043_20  var043_21  var043_22  var043_23 var043_24  var043_25  var043_26  var043_27  var043_28  var043_29  var043_30  var043_31  var043_32  var043_33
0  var043_2  var043_6  var043_5  var043_7  var043_6  var043_9  var043_4  var043_4  var043_6  var043_15  var043_15  var043_13  var043_12  var043_20  var043_20  var043_17  var043_18  var043_17  var043_18  var043_15  var043_22  var043_23  var043_22  var043_6  var043_26  var043_27  var043_26  var043_29  var043_30  var043_29  var043_30  var043_31  var043_32
2019-01-26 18:29:59,287 cache.py[62] DEBUG ====Write 33 records to File#./cache/get_corr_wtid=var043=.h5, with:df_0
2019-01-26 18:29:59,316 util_log.py[49] INFO get_corr_wtid cost  256.52 sec:(1 paras)(['var043'], []), return:DataFrame, end 


,var043_1,var043_2,var043_3,var043_4,var043_5,var043_6,var043_7,var043_8,var043_9,var043_10,var043_11,var043_12,var043_13,var043_14,var043_15,var043_16,var043_17,var043_18,var043_19,var043_20,var043_21,var043_22,var043_23,var043_24,var043_25,var043_26,var043_27,var043_28,var043_29,var043_30,var043_31,var043_32,var043_33
var043_1,1.000000,0.855251,0.811411,0.845869,0.845782,0.851014,0.845462,0.816038,0.845598,0.836170,0.833383,0.848145,0.841945,0.834861,0.838350,0.825167,0.823152,0.835330,0.836603,0.829309,0.818203,0.814421,0.813804,0.845069,0.775244,0.813909,0.819230,0.812033,0.819051,0.818406,0.807481,0.814775,0.771796
var043_2,0.855251,1.000000,0.813538,0.842588,0.854567,0.856507,0.850491,0.823601,0.839257,0.834365,0.829730,0.845929,0.832123,0.826660,0.838314,0.824962,0.827101,0.841117,0.832679,0.826695,0.821919,0.821381,0.820218,0.840588,0.786804,0.818963,0.822630,0.802526,0.819690,0.821261,0.816100,0.823486,0.782648
var043_3,0.811411,0.813538,1.000000,0.801099,0.828519,0.826580,0.808738,0.797442,0.809065,0.794587,0.792037,0.813718,0.803591,0.793295,0.805731,0.799790,0.802109,0.810991,0.801281,0.809259,0.789998,0.796681,0.795540,0.813334,0.749112,0.788163,0.793374,0.782748,0.792905,0.793164,0.786172,0.787975,0.768894
var043_4,0.845869,0.842588,0.801099,1.000000,0.850405,0.851581,0.864057,0.848429,0.838924,0.847011,0.836851,0.843253,0.838301,0.830857,0.843113,0.833973,0.831906,0.838811,0.824482,0.835596,0.827377,0.822401,0.826354,0.825445,0.791503,0.819701,0.822722,0.798069,0.816764,0.818262,0.820079,0.819482,0.788631
var043_5,0.845782,0.854567,0.828519,0.850405,1.000000,0.860965,0.851134,0.839300,0.855852,0.846914,0.842992,0.846513,0.838242,0.839716,0.851089,0.828745,0.831033,0.843775,0.831509,0.842374,0.830078,0.826087,0.826419,0.850786,0.801959,0.826120,0.821030,0.797738,0.819680,0.821827,0.822220,0.826533,0.797235
var043_6,0.851014,0.856507,0.826580,0.851581,0.860965,1.000000,0.853026,0.831612,0.864812,0.843585,0.850678,0.848027,0.842767,0.839286,0.853314,0.840026,0.838598,0.853291,0.845427,0.841834,0.836731,0.835238,0.827557,0.863271,0.803018,0.832012,0.830671,0.810275,0.835209,0.833370,0.834851,0.838241,0.795471
var043_7,0.845462,0.850491,0.808738,0.864057,0.851134,0.853026,1.000000,0.836775,0.842219,0.850039,0.849246,0.853443,0.844810,0.841723,0.849353,0.838793,0.835649,0.846265,0.834707,0.842657,0.834815,0.832267,0.832290,0.835584,0.798048,0.825971,0.826257,0.804365,0.824401,0.826301,0.825544,0.827538,0.796918
var043_8,0.816038,0.823601,0.797442,0.848429,0.839300,0.831612,0.836775,1.000000,0.818323,0.833107,0.826205,0.826571,0.816740,0.824519,0.837601,0.821998,0.813032,0.819767,0.809940,0.831023,0.815469,0.814385,0.819598,0.804594,0.787558,0.813967,0.812136,0.786533,0.808797,0.808847,0.808196,0.804809,0.790564
var043_9,0.845598,0.839257,0.809065,0.838924,0.855852,0.864812,0.842219,0.818323,1.000000,0.838131,0.840473,0.845366,0.839172,0.840883,0.853830,0.826535,0.830077,0.844376,0.832837,0.847680,0.837111,0.817703,0.826329,0.853939,0.779524,0.821988,0.817855,0.805760,0.822728,0.824114,0.833347,0.837038,0.801436
var043_10,0.836170,0.834365,0.794587,0.847011,0.846914,0.843585,0.850039,0.833107,0.838131,1.000000,0.850900,0.848142,0.838081,0.848038,0.851578,0.833012,0.827364,0.843154,0.831752,0.849860,0.841105,0.826005,0.834955,0.836602,0.791076,0.818862,0.817505,0.791785,0.812579,0.814492,0.822537,0.825637,0.809267


In [3]:



def get_train_rename(wtid, col_name, key=None):
    if key is None:
        key = wtid
    train = get_train_ex(wtid)[[col_name, 'time_sn', 'time_slot_7']]
    train.columns = [f'{col}_{key}' if 'var' in col else col for col in train.columns]
    return train


def get_train_feature_multi_file(wtid, col):

    cor = get_corr_wtid(col)
    related_wtid_list = cor[f'{col}_{wtid}'].sort_values(ascending=False)[1:3]
    related_wtid_list = [int(col.split('_')[1]) for col in related_wtid_list.index]


    train = get_train_rename(wtid, col)
    print(train.columns)
    train['id']=train.index
    for related_wtid in related_wtid_list:
        train_tmp = get_train_rename(related_wtid, col)
        train_tmp = train_tmp.drop(axis='column', columns=['time_sn'])
        train = train.merge(train_tmp, how='left', on=['time_slot_7'])
    train = train.drop_duplicates(['id'])
    train = train.set_index('id')
    col_list = [col for col in train.columns if 'var' in col]
    col_list.append('time_sn')
    train = train[col_list]
    return train
tmp = get_train_feature_multi_file(33, 'var034')
tmp


2019-01-26 20:38:25,208 util_log.py[41] INFO get_corr_wtid begin with(1 paras) :['var034'], []
2019-01-26 20:38:25,216 cache.py[38] DEBUG Can not find cache from file:./cache/get_corr_wtid=var034=.h5
/Users/lali2/Documents/workspace_py/df_jf/core/feature.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  template.drop(columns='sn', errors='ignore', inplace=True)
2019-01-26 20:38:31,819 feature.py[96] DEBUG Convert train#1 from (352124, 70) to (359846, 69)
/Users/lali2/Documents/workspace_py/df_jf/core/feature.py:462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train['index'] = train.index
201

2019-01-26 20:42:38,822 cache.py[62] DEBUG ====Write 33 records to File#./cache/get_corr_wtid=var034=.h5, with:df_0
2019-01-26 20:42:38,848 util_log.py[49] INFO get_corr_wtid cost  253.64 sec:(1 paras)(['var034'], []), return:DataFrame, end 
/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index(['var034_33', 'time_sn', 'time_slot_7'], dtype='object')


,var034_33,var034_32,var034_31,time_sn
id,,,,
0,134.81,184.28,230.29,3.165
1,153.99,164.81,200.32,10.588
2,145.61,184.28,158.58,17.953
3,134.81,184.28,186.35,25.339
4,153.99,184.28,200.32,32.713
5,145.61,151.61,186.35,40.072
6,124.80,134.90,230.29,47.438
7,103.94,151.61,NaN,54.824
8,134.81,170.16,266.89,62.234


In [ ]:

print(cor.loc[f'{col_name}_1', f'{col_name}_2'])


step = 5000
begin = 4000 + step
end = 6000 +step



print(len(train_1.time_sn_1.loc[begin:end]) , len(train_2.time_sn_2.loc[begin:end]))

train_1[f'{col_name}_1'].loc[begin:end].plot()

train_2[f'{col_name}_2'].loc[begin:end].plot()

plt.show()

### 

In [ ]:
train_1.loc[begin:end]

In [ ]:
train_1.var034_1.iloc[begin:end].plot?

In [ ]:
train_1.var034_1.iloc[begin:end].plot

In [ ]:
tmp = std.groupby(['col', 'data_type'])['mean'].mean().sort_values().to_frame().reset_index()
print(tmp.columns)
tmp[tmp['data_type']=='int16']

In [ ]:
columns = list(date_type.keys())
columns.remove('wtid')
columns = sorted(columns)
for wtid in sorted(range(1, 34), reverse=True):
    for col in columns:
        check_std(wtid,col)

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)


get_sub_template()

In [ ]:
get_blocks().iloc[45]

In [ ]:
get_blocks().iloc[6735]

In [ ]:

tmp = get_train_ex(3).loc[173119-4000:173119][['time_sn', 'var002']]
tmp.loc[(tmp.time_sn > 1250000) & (tmp.time_sn < 1290000)]

In [ ]:


feature_list = get_train_feature(1, 'var001')

len(feature_list)

In [ ]:
bk.groupby(['wtid','col']).length.sum()

In [ ]:
template = get_train_ex(1)
template.ts = pd.to_datetime(template.ts)
template['ts2'] = template.ts.shift(1)
template['gap'] = template.ts - template.ts2

template.gap = template['gap'].astype(int)//1000000000 
template['gap'].astype(int).hist()
plot.show()

In [ ]:
template['gap'].value_counts().sort_index()
#template['gap'].drop_duplicates().sort_values()

In [ ]:
 
        
col = 'var003'
get_result(1, col)


In [ ]:
train = get_train_ex(1)
train.iloc[354983-3: 354983+3]

In [ ]:

print(__doc__)

# Authors: Jan Hendrik Metzen <jhm@informatik.uni-bremen.de>
#
# License: BSD 3 clause

import numpy as np

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel


rng = np.random.RandomState(0)
X = rng.uniform(0, 5, 20)[:, np.newaxis]
y = 0.5 * np.sin(3 * X[:, 0]) + rng.normal(0, 0.5, X.shape[0])

# First run
plt.figure(0)
kernel = 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) \
    + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=0.0).fit(X, y)
X_ = np.linspace(0, 5, 100)
y_mean, y_cov = gp.predict(X_[:, np.newaxis], return_cov=True)
plt.plot(X_, y_mean, 'k', lw=3, zorder=9)
plt.fill_between(X_, y_mean - np.sqrt(np.diag(y_cov)),
                 y_mean + np.sqrt(np.diag(y_cov)),
                 alpha=0.5, color='k')
plt.plot(X_, 0.5*np.sin(3*X_), 'r', lw=3, zorder=9)
plt.scatter(X[:, 0], y, c='r', s=50, zorder=10, edgecolors=(0, 0, 0))
plt.title("Initial: %s\nOptimum: %s\nLog-Marginal-Likelihood: %s"
          % (kernel, gp.kernel_,
             gp.log_marginal_likelihood(gp.kernel_.theta)))
plt.tight_layout()

# Second run
plt.figure(1)
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3)) \
    + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e+1))
gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=0.0).fit(X, y)
X_ = np.linspace(0, 5, 100)
y_mean, y_cov = gp.predict(X_[:, np.newaxis], return_cov=True)
plt.plot(X_, y_mean, 'k', lw=3, zorder=9)
plt.fill_between(X_, y_mean - np.sqrt(np.diag(y_cov)),
                 y_mean + np.sqrt(np.diag(y_cov)),
                 alpha=0.5, color='k')
plt.plot(X_, 0.5*np.sin(3*X_), 'r', lw=3, zorder=9)
plt.scatter(X[:, 0], y, c='r', s=50, zorder=10, edgecolors=(0, 0, 0))
plt.title("Initial: %s\nOptimum: %s\nLog-Marginal-Likelihood: %s"
          % (kernel, gp.kernel_,
             gp.log_marginal_likelihood(gp.kernel_.theta)))
plt.tight_layout()

# Plot LML landscape
plt.figure(2)
theta0 = np.logspace(-2, 3, 49)
theta1 = np.logspace(-2, 0, 50)
Theta0, Theta1 = np.meshgrid(theta0, theta1)
LML = [[gp.log_marginal_likelihood(np.log([0.36, Theta0[i, j], Theta1[i, j]]))
        for i in range(Theta0.shape[0])] for j in range(Theta0.shape[1])]
LML = np.array(LML).T

vmin, vmax = (-LML).min(), (-LML).max()
vmax = 50
level = np.around(np.logspace(np.log10(vmin), np.log10(vmax), 50), decimals=1)
plt.contour(Theta0, Theta1, -LML,
            levels=level, norm=LogNorm(vmin=vmin, vmax=vmax))
plt.colorbar()
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Length-scale")
plt.ylabel("Noise-level")
plt.title("Log-marginal-likelihood")
plt.tight_layout()

plt.show()


In [ ]:
tmp['time_sn'] = (tmp.ts - pd.to_datetime('2018-07-01')).astype(int)/1000000000
 
# from datetime import timedelta
# tmp['gap'] = tmp['gap']/timedelta(hours=1)
# #tmp['gap'] = (  pd.to_datetime(tmp.ts2) - pd.to_datetime(tmp.ts) ) #/ np.timedelta64(1, 's')
# tmp.head()
#tmp.gap.astype(int)
tmp

In [ ]:
train = get_train_ex(1) 



missing = train[pd.isna(train['var001'])].index


In [ ]:
tmp.iloc[170590: 170599]

In [ ]:
tmp_1.var001.iloc[200:300].plot()
plot.show()

In [ ]:
tmp_1.var001.dropna().plot()
plot.show()

In [ ]:
len(tmp_1.var003), len(tmp_1.var003.dropna())

In [ ]:
from core.config import date_type
example = pd.read_csv('./input/submit_example.csv', dtype=date_type) 
example.head()


example.var016.drop_duplicates()


In [ ]:
pd.value_counts(example.var016)

In [ ]:
example.var001.iloc[0:3].astype(str)

In [ ]:
example.var001.iloc[0:3].apply(lambda val : True if val==405.37 else False)

In [ ]:
example.head()

In [ ]:
plt.show()

In [ ]:
from core import feature

In [ ]:

template = pd.read_csv('./input/template_submit_result.csv') 

print(template.shape)
#template 

In [ ]:

template = pd.read_csv('./input/template_submit_result.csv') 

train = pd.read_csv('./input/001/201807.csv') 
print(train.shape)

template = template.set_index(['ts', 'wtid'])
train = train.set_index(['ts', 'wtid'])


train = train[train.index.isin(template.index)]

train.shape

tmp = template.combine_first(train)
print(tmp.shape)


In [ ]:

 

tmp = get_sub_template()

tmp.head()

In [ ]:
tmp = get_sub_template()


In [ ]:
tmp.head(100)

In [ ]:
train_01 = pd.read_csv('./input/001/201807.csv') 
train_01.head()
#del train_01




In [ ]:
from core.feature import *
df = get_missing_analysis()

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.var001[:1].describe()

In [ ]:
df.var001.astype(int).std()

In [ ]:
np.finfo(np.float16).max

In [ ]:

def get_analysis_enum():
    col_list = ['wtid','var053','var066','var016','var020','var047',  ]

    train_list = []
    for wtid in range(1, 34):
        wtid = str(wtid)
        train = pd.read_csv(f"./input/{wtid.rjust(3,'0')}/201807.csv", usecols=col_list)
        train = train.groupby(col_list).agg({'wtid':'count'})
        train.rename(index=str, columns={"wtid": "count"}, inplace=True)
        train = train.reset_index()
        print(train.shape)
        train_list.append(train)

    all = pd.concat(train_list)
    return all

enum_type = get_analysis_enum()

In [ ]:
tmp = enum_type[['var053','var066','var016','var020','var047',]].drop_duplicates(['var053','var066','var016','var020','var047',])

In [ ]:
enum_type.sort_values(['wtid', 'count', ], ascending=False)

In [ ]:
for i in sorted((range(1, 34)),reverse=True):
    print(i)

In [ ]:
import pprint
pprint.pprint(abc)

pprint.PrettyPrinter?

In [ ]:
abc = {'a':'b', 'c':'d'}


In [ ]:
tmp = get_blocks()

In [ ]:
time_all = pd.DataFrame()
time_list = []
for i in range(1, 4):
    train = get_train_ex(i)
    print(len(train))
    time_list.append(train.ts)
    
time_all = pd.concat(time_list, axis=1)

In [ ]:
time_all['min'] = time_all.iloc[:,:-3].min(axis=1)
time_all['max'] = time_all.iloc[:,:-3].max(axis=1)
time_all['gap'] = time_all['max'] - time_all['min']
time_all['gap'] =  time_all['gap'].astype(int).fillna(0)//1000000000
time_all.head()